# Upravljanje z datotekami


## Managing Files and Directories

[File and Directory Access](https://docs.python.org/3/library/filesys.html)

Avtomatizacija taskov, kdaj se splača: [slika](https://xkcd.com/1205/)

### General OS operations

In [ ]:
import os

- `os.getcwd()`: Return a string representing the current working directory.

In [ ]:
os.getcwd()

'/home/jovyan/work/osnovni_tecaj/10_Interact_with_the_Operating_System'

- `os.chdir(path)` Change the current working directory to path.

In [ ]:
os.chdir('./data')

- `os.path.exists(path)`: Return True if path refers to an existing path or an open file descriptor. Returns False for broken symbolic links. On some platforms, this function may return False if permission is not granted to execute os.stat() on the requested file, even if the path physically exists.

In [ ]:
os.path.exists('data/')

True

- `os.path.abspath(path)`: Return a normalized absolutized version of the pathname path. On most platforms, this is equivalent to calling the function normpath() as follows: normpath(join(os.getcwd(), path)).

In [ ]:
os.path.abspath("data/example.txt")

'/home/jovyan/work/osnovni_tecaj/10_Interact_with_the_Operating_System/data/example.txt'

- `os.path.getsize(path)`: Return the size, in bytes, of path. Raise OSError if the file does not exist or is inaccessible.

In [ ]:
os.path.getsize('data/weblog.csv')

1115343

- `os.path.isdir(path)`: Return True if path is an existing directory. This follows symbolic links, so both islink() and isdir() can be true for the same path.

In [ ]:
os.path.isdir('data')

True

- `os.path.isfile(path)`: Return True if path is an existing directory. This follows symbolic links, so both islink() and isdir() can be true for the same path.

In [ ]:
os.path.isfile('data/example3.txt')

True

- `os.path.join(path, *paths)`: Join one or more path components intelligently. The return value is the concatenation of path and any members of *paths with exactly one directory separator (os.sep) following each non-empty part except the last, meaning that the result will only end in a separator if the last part is empty. If a component is an absolute path, all previous components are thrown away and joining continues from the absolute path component.

In [ ]:
os.path.join('/data', 'examples/test.py')

'/data/examples/test.py'

- `os.path.getatime(path)`: Return the time of last access of path. The return value is a floating point number giving the number of seconds since the epoch (see the time module). Raise OSError if the file does not exist or is inaccessible.
- `os.path.getmtime(path)`: Return the time of last modification of path. The return value is a floating point number giving the number of seconds since the epoch (see the time module). Raise OSError if the file does not exist or is inaccessible.
- `os.path.getctime(path)`: Return the system’s ctime which, on some systems (like Unix) is the time of the last metadata change, and, on others (like Windows), is the creation time for path. The return value is a number giving the number of seconds since the epoch (see the time module). Raise OSError if the file does not exist or is inaccessible.

In [ ]:
timestamp = os.path.getmtime('data/example.txt')
print(timestamp)

1580161039.466197


In [ ]:
# pretvorba v datetime
from datetime import datetime

def unix_to_str_time(unix_time, time_format='%Y-%m-%d %H:%M:%S'):
    return datetime.utcfromtimestamp(unix_time).strftime(time_format)

In [ ]:
unix_to_str_time(timestamp)

'2020-01-27 21:37:19'

### Making Directories

#### Creating a Single Directory

In [ ]:
import os

os.mkdir('data/example_directory/')

In [ ]:
os.mkdir('data/example_directory/')

FileExistsError: [Errno 17] File exists: 'data/example_directory/'

In [ ]:
try:
    os.mkdir('data/example_directory/')
except FileExistsError as exc:
    print(exc)

[Errno 17] File exists: 'data/example_directory/'


#### Creating Multiple Directories

In [ ]:
os.makedirs('data/leto/mesec/dan')

In [ ]:
os.makedirs('data/leto/2018/10/05', mode=0o770)

[Premissions](https://danielmiessler.com/images/permissions.png)

### Getting a Directory Listing

- [os.listdir](https://docs.python.org/3/library/os.html#os.listdir)
- [os.scandir](https://docs.python.org/3/library/os.html#os.scandir)
    

> The scandir() function returns directory entries along with file attribute information, giving better performance for many common use cases.

In [ ]:
# Directory Listing in Legacy Python Versions
import os
entries = os.listdir('./')
print(entries)

['Command_line_with_python.ipynb', 'skripte', '.ipynb_checkpoints', 'data']


In [ ]:
# Primer: kako vidimo katero so mape katero datoteke
directory = './data'

for name in os.listdir(directory):
    fullname = os.path.join(directory, name) #joinmao ne glede na sistem
    if os.path.isdir(fullname):
        print(f'{fullname} is a directory')
    else:
        print(f'{fullname} is a file')

./data/newdir is a directory
./data/example.txt is a file
./data/.ipynb_checkpoints is a directory


In [ ]:
# Directory Listing in Modern Python Versions
import os
entries = os.scandir('./')
print(entries)

In [ ]:
import os

with os.scandir('./') as entries:
    for entry in entries:
        print(entry.name)

Command_line_with_python.ipynb
skripte
.ipynb_checkpoints
data


In [ ]:
# Primer
with os.scandir('./') as it:
    for entry in it:
        if not entry.name.startswith('.') and entry.is_file():
            print(entry.name)

Command_line_with_python.ipynb


In [ ]:
# Listing Subdirectories
# List all subdirectories using scandir()
basepath = './'
with os.scandir(basepath) as entries:
    for entry in entries:
        if entry.is_dir():
            print(entry.name)

skripte
.ipynb_checkpoints
data


### Getting File Attributes

In [ ]:
import os
with os.scandir('./') as dir_contents:
    for entry in dir_contents:
        info = entry.stat()
        #print(info)
        print(f'Name: {entry.name}, Time: {info.st_mtime}s')

Name: Command_line_with_python.ipynb, Time: 1580251977.5320895s
Name: skripte, Time: 1580242340.9039881s
Name: .ipynb_checkpoints, Time: 1579434568.7935946s
Name: data, Time: 1580162510.3971303s


[os.stat_result](https://docs.python.org/3/library/os.html#os.stat_result)

In [ ]:
from datetime import datetime
from os import scandir

def convert_date(timestamp):
    d = datetime.utcfromtimestamp(timestamp)
    formated_date = d.strftime('%Y-%m-%d %H:%M:%S')
    return formated_date

def get_files(dir_path):
    dir_entries = scandir(dir_path)
    for entry in dir_entries:
        if entry.is_file():
            info = entry.stat()
            # The width specifier sets the width of the value. 
            print(f'{entry.name:20}\t Last Modified: {convert_date(info.st_mtime)}')
            
get_files('./skripte')

example04.py        	 Last Modified: 2020-01-19 16:26:12
example00.py        	 Last Modified: 2020-01-19 14:46:31
example03.py        	 Last Modified: 2020-01-19 14:50:58
create_file.py      	 Last Modified: 2020-01-28 19:48:09
host.py             	 Last Modified: 2020-01-28 21:20:55
healthcheck_script.py	 Last Modified: 2020-01-27 20:41:51
example01.py        	 Last Modified: 2020-01-19 14:43:19
example05.py        	 Last Modified: 2020-01-25 23:07:59
example02.py        	 Last Modified: 2020-01-19 14:43:03


### Filename Pattern Matching

    mkdir some_directory
    cd some_directory/
    mkdir sub_dir
    touch sub_dir/file1.py sub_dir/file2.py
    touch data_{01..03}.txt data_{01..03}_backup.txt admin.py tests.py

#### Using String Methods

In [ ]:
import os

# Get .txt files
def search_file_by_extension(directory, extension):
    for f_name in os.listdir(directory):
        if f_name.endswith(extension):
            print(f_name)

In [ ]:
search_file_by_extension('data/some_directory/', '.txt')

data_01_backup.txt
data_02_backup.txt
data_01.txt
data_03_backup.txt
data_03.txt
data_02.txt


#### Filename Pattern Matching Using fnmatch

> [fnmatch — Unix filename pattern matching](https://docs.python.org/3.8/library/fnmatch.html)

In [ ]:
import os
import fnmatch

for file_name in os.listdir('data/some_directory/'):
    if fnmatch.fnmatch(file_name, '*.txt'):
        print(file_name)

data_01_backup.txt
data_02_backup.txt
data_01.txt
data_03_backup.txt
data_03.txt
data_02.txt


In [ ]:
for filename in os.listdir('data/some_directory/'):
    if fnmatch.fnmatch(filename, 'data_*_backup.txt'):
        print(filename)

data_01_backup.txt
data_02_backup.txt
data_03_backup.txt


#### Filename Pattern Matching Using glob

> [glob — Unix style pathname pattern expansion](https://docs.python.org/3.8/library/glob.html?highlight=glob#module-glob)

In [ ]:
import glob
glob.glob('data/some_directory/sub_dir/*.py')

['data/some_directory/sub_dir/file1.py',
 'data/some_directory/sub_dir/file2.py']

In [ ]:
import glob
for name in glob.glob('data/some_directory/*[0-9]*.txt'):
    print(name)

data/some_directory/data_01_backup.txt
data/some_directory/data_02_backup.txt
data/some_directory/data_01.txt
data/some_directory/data_03_backup.txt
data/some_directory/data_03.txt
data/some_directory/data_02.txt


In [ ]:
import glob
for file in glob.iglob('data/some_directory/**/*.py', recursive=True):
    print(file)

data/some_directory/tests.py
data/some_directory/admin.py
data/some_directory/sub_dir/file1.py
data/some_directory/sub_dir/file2.py


### Traversing Directories and Processing Files

In [ ]:
# Walking a directory tree and printing the names of the directories and files
for dirpath, dirnames, files in os.walk('./data/some_directory/'):
    print(f'Found directory: {dirpath}')
    for file_name in files:
        print(file_name)

Found directory: ./data/some_directory/
tests.py
data_01_backup.txt
data_02_backup.txt
admin.py
data_01.txt
data_03_backup.txt
data_03.txt
data_02.txt
Found directory: ./data/some_directory/sub_dir
file1.py
file2.py


In [ ]:
# vaja dodamo v vsak file neko vsebino
for dirpath, dirnames, files in os.walk('./data/some_directory/'):
    print(f'Found directory: {dirpath}')
    for file_name in files:
        fullname = os.path.join(dirpath, file_name)
        print(f'Editing {fullname}...')
        with open(fullname, 'w') as f:
            f.write('Heloo')

Found directory: ./data/some_directory/
Editing ./data/some_directory/tests.py...
Editing ./data/some_directory/data_01_backup.txt...
Editing ./data/some_directory/data_02_backup.txt...
Editing ./data/some_directory/admin.py...
Editing ./data/some_directory/data_01.txt...
Editing ./data/some_directory/data_03_backup.txt...
Editing ./data/some_directory/data_03.txt...
Editing ./data/some_directory/data_02.txt...
Found directory: ./data/some_directory/sub_dir
Editing ./data/some_directory/sub_dir/file1.py...
Editing ./data/some_directory/sub_dir/file2.py...


### Deleting Files and Directories

#### Deleting Files in Python

In [ ]:
import os

data_file = './data/delete.txt'
os.remove(data_file)

In [ ]:
import os

data_file = './data/delete.txt'
os.unlink(data_file)

In [ ]:
import os

data_file = './data/delete.txt'

# If the file exists, delete it
if os.path.isfile(data_file):
    os.remove(data_file)
else:
    print(f'Error: {data_file} not a valid filename')

Error: ./data/delete.txt not a valid filename


In [ ]:
import os

data_file = './data/delete.txt'

# Use exception handling
try:
    os.remove(data_file)
except OSError as e:
    print(f'Error: {data_file} : {e.strerror}')

Error: ./data/delete.txt : No such file or directory


#### Deleting Directories

In [ ]:
import os

trash_dir = './data/test'

try:
    os.rmdir(trash_dir)
except OSError as e:
    print(f'Error: {trash_dir} : {e.strerror}')

Here, the trash_dir directory is deleted by passing its path to os.rmdir(). If the directory isn’t empty, an error message is printed to the screen:

    Traceback (most recent call last):
      File '<stdin>', line 1, in <module>
    OSError: [Errno 39] Directory not empty: 'my_documents/bad_dir'

#### Deleting Entire Directory Trees

In [ ]:
!mkdir ./data/test
!touch ./data/test/lalal

In [ ]:
import shutil

trash_dir = './data/test'

try:
    shutil.rmtree(trash_dir)
except OSError as e:
    print(f'Error: {trash_dir} : {e.strerror}')

In [ ]:
import os

for dirpath, dirnames, files in os.walk('.', topdown=False):
    try:
        os.rmdir(dirpath)
    except OSError as ex:
        pass

<div class="table-responsive">
<table class="table table-hover">
<thead>
<tr>
<th>Function</th>
<th>Description</th>
</tr>
</thead>
<tbody>
<tr>
<td><code>os.remove()</code></td>
<td>Deletes a file and does not delete directories</td>
</tr>
<tr>
<td><code>os.unlink()</code></td>
<td>Is identical to <code>os.remove()</code> and deletes a single file</td>
</tr>
<tr>
<td><code>pathlib.Path.unlink()</code></td>
<td>Deletes a file and cannot delete directories</td>
</tr>
<tr>
<td><code>os.rmdir()</code></td>
<td>Deletes an empty directory</td>
</tr>
<tr>
<td><code>pathlib.Path.rmdir()</code></td>
<td>Deletes an empty directory</td>
</tr>
<tr>
<td><code>shutil.rmtree()</code></td>
<td>Deletes entire directory tree and can be used to delete non-empty directories</td>
</tr>
</tbody>
</table>
</div>

### Copying, Moving, and Renaming Files and Directories

#### Copying Files in Python

In [ ]:
import shutil

src = './data/example.txt'
dst = './data/example2.txt'
shutil.copy(src, dst)

'./data/example2.txt'

In [ ]:
import shutil

src = './data/example.txt'
dst = './data/example3.txt'
shutil.copy2(src, dst)

'./data/example3.txt'

#### Copying Directories

In [ ]:
import shutil
shutil.copytree('data_1', 'data1_backup')

#### Moving Files and Directories

In [ ]:
import shutil
shutil.move('dir_1/', 'backup/')

#### Renaming Files and Directories

In [ ]:
os.rename('first.zip', 'first_01.zip')

### Archiving

#### ZIP Files

[zipfile — Work with ZIP archives](https://docs.python.org/3/library/zipfile.html?highlight=zip#module-zipfile)

#### TAR Archives

[tarfile — Read and write tar archive files](https://docs.python.org/3/library/tarfile.html?highlight=tar#module-tarfile)

#### An Easier Way of Creating Archives

In [ ]:
import shutil

# shutil.make_archive(base_name, format, root_dir)
shutil.make_archive('./data/leto', 'tar', './data/leto')

'/home/jovyan/work/osnovni_tecaj/10_Interact_with_the_Operating_System/data/leto.tar'

In [ ]:
shutil.unpack_archive('./data/leto.tar', 'data/extract_dir/')

# Razlaga

In [3]:
import os
from pathlib import Path


In [2]:
os.getcwd()

'c:\\LEON\\ICTA\\Tečaji\\icta-python-osnovni-public\\14_Delo_z_datotekami'

In [7]:
# my_path = Path(__file__)
my_path = Path(r"C:\LEON\ICTA\Tečaji\icta-python-osnovni-public\00_Playground\project_01\main.py")


WindowsPath('C:/LEON/ICTA/Tečaji/icta-python-osnovni-public/00_Playground/project_01/data')

In [17]:
my_path.parent / "data"

WindowsPath('C:/LEON/ICTA/Tečaji/icta-python-osnovni-public/00_Playground/project_01/data')

In [23]:
my_path = Path(r"C:\LEON\ICTA\Tečaji\icta-python-osnovni-public\00_Playground\project_01\main.py")
my_path = my_path.parent.joinpath("data")

with os.scandir(my_path) as dir_contents:
    for entry in dir_contents:
        info = entry.stat()
        # print(info)
        print(f"Name: {entry.name}, Time: {info.st_mtime}s, Size: {info.st_size}b, Is file: {entry.is_file()}")
        print(entry.path)


Name: 2_test_report_02_UBL500_20109to20110_02.01.03_905587.xlsx, Time: 1700478736.474779s, Size: 1403744b, Is file: True
C:\LEON\ICTA\Tečaji\icta-python-osnovni-public\00_Playground\project_01\data\2_test_report_02_UBL500_20109to20110_02.01.03_905587.xlsx
Name: memoryLogCommands_02_01_09.xml, Time: 1700479821.7023146s, Size: 346340b, Is file: True
C:\LEON\ICTA\Tečaji\icta-python-osnovni-public\00_Playground\project_01\data\memoryLogCommands_02_01_09.xml
Name: OutputLog_COM51_20231114-160929.log, Time: 1700478736.4385574s, Size: 6877b, Is file: True
C:\LEON\ICTA\Tečaji\icta-python-osnovni-public\00_Playground\project_01\data\OutputLog_COM51_20231114-160929.log
Name: white-60-wp-0703-14-01.txt, Time: 1700478736.5372758s, Size: 2798110b, Is file: True
C:\LEON\ICTA\Tečaji\icta-python-osnovni-public\00_Playground\project_01\data\white-60-wp-0703-14-01.txt
